In [1]:
from datetime import datetime
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import preprocessor as p
import os
import time


In [2]:
consumer_key = "P5i8oS0DWMlplkEPWHewdcvnZ"
consumer_secret = "opXG0LpwkrMCPjogMdTikFXWbbvI2tnKFglG1YOc1BQj04Uvan"
access_token = "4091794879-nfBkiutGX2qkiVbETwsxOXENrk4ALrqV8keqSG4"
access_token_secret = "bZDq7LGETyqILIogC4IOSDMkaQKjYdc3wSEBJ3r2NgZsJ"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)



In [3]:
username = '_fertileground_'
count = 1000
try:
    
    tweets = tweepy.Cursor(api.user_timeline, id=username, tweet_mode='extended').items(count)

    
    tweets_list = [[tweet.created_at, tweet.id, tweet.full_text, tweet.user.location]
                   for tweet in tweets]

   
    tweets_df = pd.DataFrame(tweets_list)
except BaseException as e:
    print('failed on_status,', str(e))
    time.sleep(3)

tweets_df.columns = ['Tweet Date', 'Tweet ID', 'Tweet text', 'Tweet Location']
print(tweets_df)
#tweets_df.to_csv('fertilegroundtweets.csv')

             Tweet Date             Tweet ID  \
0   2021-03-18 18:21:57  1372614266386329600   
1   2021-03-18 02:28:27  1372374308786466824   
2   2021-03-16 15:59:11  1371853563475988482   
3   2021-03-11 23:04:26  1370148640539430919   
4   2021-03-11 23:02:15  1370148088678060037   
..                  ...                  ...   
562 2019-06-19 21:08:20  1141452680130420738   
563 2019-06-19 21:04:46  1141451780070477826   
564 2019-06-19 20:49:10  1141447855363829760   
565 2019-03-21 21:07:00  1108837436958953473   
566 2019-02-02 16:49:42  1091740454146723841   

                                            Tweet text Tweet Location  
0    RT @NFUDC: When restaurants closed during the ...    Jackson, MS  
1    RT @cadwego: “Even as someone who is fairly cr...    Jackson, MS  
2    RT @BloombergDotOrg: We believe in the power o...    Jackson, MS  
3    Thank you for supporting our local community i...    Jackson, MS  
4    RT @MikeBloomberg: Many would like to forget t...    Jacks

In [4]:
#for different search terms and to get the reply count, retweet counts and other data from the tweets related to the search term.

import snscrape.modules.twitter as sntwitter


tweets_list2 = []



terms=['fertile ground', 'food access', 'food insecurity', 'food security', 'food health', 'food deserts', 'food swamps','#fertilegroundjxn','#healthyfood']

# Using TwitterSearchScraper  to scrape data and append tweets to list
for term in terms:
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(term+' geocode:32.2988,-90.1848,50km since:2018-03-01').get_items()):
   
        if i>500:
            break
      
        tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username, tweet.user.location, tweet.replyCount, tweet.retweetCount, tweet.likeCount, tweet.quoteCount])
    
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username','User Location','Reply Count','Retweet Count','Like Count','Quote Count'])
tweets_df2.drop_duplicates(subset="Tweet Id", keep=False, inplace=True)
tweets_df2

,Datetime,Tweet Id,Text,Username,User Location,Reply Count,Retweet Count,Like Count,Quote Count
0,2021-01-22 04:12:08+00:00,1352469068452868103,@GanucheauAdam Thank you for retweeting! Also ...,_fertileground_,"Jackson, MS",0,0,0,0
1,2021-01-22 01:00:14+00:00,1352420775530291202,We’ve BEEN asking for this!\n\nKey House leade...,_fertileground_,"Jackson, MS",0,0,0,0
2,2020-11-30 22:02:50+00:00,1333531965245566977,Mississippi Goals! https://t.co/DrQE0Ycn3j,_fertileground_,"Jackson, MS",0,0,2,0
3,2020-11-30 22:01:50+00:00,1333531710907150336,Tips! https://t.co/iyte1eETRN,_fertileground_,"Jackson, MS",0,0,0,0
4,2020-11-18 12:44:34+00:00,1329042818199908353,@JacksonStateU Athletics and @UHC will be prov...,_fertileground_,"Jackson, MS",0,0,0,0
...,...,...,...,...,...,...,...,...,...
149,2018-07-17 18:54:06+00:00,1019294187592802309,#transformationtuesday #newbody #healthylifest...,epicdanceandfit,"Ridgeland, MS",0,0,1,0
150,2018-07-14 02:40:19+00:00,1017961962251186177,#postrunfuel #berries #strawberries #strawberr...,SigChiPsi,"ÜT: 31.608732,-94.600016",0,0,0,0
151,2018-07-11 02:06:15+00:00,1016866225324949505,#postrunmeal #crabsalad #and #sweetpotato #org...,SigChiPsi,"ÜT: 31.608732,-94.600016",0,0,0,0
152,2018-06-27 02:36:25+00:00,1011800386062209025,#postrunmeal #healthyfood #foodporn #foodporno...,SigChiPsi,"ÜT: 31.608732,-94.600016",0,0,0,0


In [9]:
#For news data

from GoogleNews import GoogleNews
from newspaper import Article
import pandas as pd
from newspaper import Article
from newspaper import Config
import nltk


googlenews=GoogleNews(start='03/01/2018',end='03/21/2021')
googlenews.search('food insecurity Mississippi')
result=googlenews.result()
df=pd.DataFrame(result)

for i in range(2,15):
    googlenews.getpage(i)
    result=googlenews.result()
    df=pd.DataFrame(result)
user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'
config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 50

list=[]
for ind in df.index:
    try:
        dict={}
        article = Article(df['link'][ind],config=config)
        article.download()
        article.parse()
        article.nlp()
        dict['Date']=df['date'][ind]
        dict['Media']=df['media'][ind]
        dict['Title']=article.title
        dict['Article']=article.text
        dict['Summary']=article.summary
        list.append(dict)
    except:
        print("caught a timeout")
news_df=pd.DataFrame(list)
googlenews.clear()

news_df

caught a timeout
caught a timeout
caught a timeout


C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname MST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


caught a timeout
caught a timeout
caught a timeout
caught a timeout


C:\Users\rishabhstha\AppData\Local\Programs\Python\Python38\lib\site-packages\dateutil\parser\_parser.py:1213: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


,Date,Media,Title,Article,Summary
0,"Oct 10, 2020",Mississippi State University,MSU’s Keenum applauds selection of World Food ...,Contact: James Carskadon\n\nMSU President Mark...,"(Submitted Photo)STARKVILLE, Miss.— Mississipp..."
1,"Dec 17, 2020",WJTV,Mississippi Food Network helps with food insec...,"JACKSON, Miss. (WJTV) – The coronavirus pandem...",(WJTV) – The coronavirus pandemic has impacted...
2,"Feb 26, 2020",Jackson Free Press,Jackson's Food Insecurity Focus in April Expo,In a state where one in four children go to be...,The U.S. Department of Agriculture defines foo...
3,"Nov 23, 2020",,"Survey: Food Insecurity, Lack of Support Remai...","JACKSON, Mississippi - Since the start of the ...","JACKSON, Mississippi - Since the start of the ..."
4,"May 4, 2018",Mississippi Today,Mississippi still the hungriest state,"For the eighth straight year, Mississippi has ...",This is the eighth edition of Feeding America’...
...,...,...,...,...,...
128,"Aug 21, 2020",WCBI,Food insecurity remains a concern for many fam...,"During this time of uncertainty, dozens of fam...","During this time of uncertainty, dozens of fam..."
129,"Nov 25, 2020",WVEC,Meal demand spiking: Newport News organization...,"Find out where you can get three free, hot mea...","NEWPORT NEWS, Va. — Hampton Roads nonprofits a..."
130,"Mar 18, 2020",Jackson Free Press,Jackson Free Press,"Sandra Shelson has served the Jackson area, an...","Sandra Shelson has served the Jackson area, an..."
131,"Dec 2, 2020",NBC News,"Police in Jackson, Mississippi, want access to...","JACKSON, Miss. — Strapped for cash and facing ...","Other Fusus clients, including police departme..."
